# TRADUCCIÓN DE TEXTOS

Este cuaderno de Jupyter está dedicado a como se ha realizado la función que traduce los textos de las diferentes redes sociales y se introducen en una nueva columna. <br>

Esto es necesario debido a que no todos los textos se deben de encontrar en los idiomas deseados ya que España cuenta con diferentes dialectos y alguas publicaciones son en otros idiomas.<br>

Los idiomas que interesan y que a los que se van a traducir los textos son el español y el ingles

# PSEUDOCÓDIGO

Lo primero de todo es crear el pseudocódigo para entender la función de una manera más humana.

Se conecta a la base de datos y se extrae la tabla<br>
Se extrae la columna del texto a traducir<br>
Para cada uno de los textos recogidos:<br>
&emsp; Identificar idioma<br>
&emsp;si es español<br>
&emsp; &emsp; traducir al ingles<br>
&emsp;si es ingles<br>
&emsp; &emsp; traducir al espñol<br>
&emsp; &emsp; si no es ninguno <br>
&emsp; &emsp; traducir al ingles y al español<br>

Introducir datos <br>

# LIBRERÍAS

Para la correcta extracción y recopilación de los datos, se aplicarán diferentes librerías pero la librería principal es:<br><br>
**__DEEP_TRANSLATOR__**: Esta librería es una de las librerías de Python con mejor relación velocidad precisón debido a que está conectada a la API del traductor de Google y puede detectar mas de 27 idiomas.  __[aquí](https://pypi.org/project/deep-translator/)__.<br><br>




Otras librerías que se usan son:
- **Pandas** : Manipulación de datos
- **Mysql.connector**: Conexión con la base de datos
- **Codecs**: Normalización de caracteres
- **TQDM**: Muestra barra de progreso

Una vez todo explicado se empieza importando las librerías a utilizar

In [ ]:
import mysql.connector
import codecs
import pandas as pd
from tqdm import tqdm
from deep_translator import GoogleTranslator
from language_detector import detect_language


Una vez se tienen todas la librerías se empieza a codificar, importando los datos desde sql, habiendo realizado la conexion previamente

In [ ]:

codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)

database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

In [ ]:
df = pd.read_sql("SELECT * FROM tweets",database)
contenido = df["contenido"].tolist()
contenido2 = df["contenido_raw"].tolist()
idioma = df["idioma"].tolist()

In [ ]:
from deep_translator import GoogleTranslator
from language_detector import detect_language


# - - Esta parte del código traduce únicamente un texto plano
traductor_espanol = GoogleTranslator(source='auto', target='es')
traductor_ingles = GoogleTranslator(source='auto', target='en')
# sqlinsert = "ALTER TABLE tweets ADD contenido_espanol TEXT"
# cursor.execute(sqlinsert)
# sqlinsert2 = "ALTER TABLE tweets ADD contenido_ingles TEXT"
# cursor.execute(sqlinsert2)
texto_espanol = [] 
texto_ingles = []
for z in tqdm(range(len(contenido))):

    if idioma [z] != "es":
        if detect_language(contenido2[z]) == "Spanish":
            traducido_espanol = contenido2[z]
        else:
            try:   
                traducido_espanol = traductor_espanol.translate(text = contenido[z])
            except:
                traducido_espanol = None
                print("error")

    else:
        traducido_espanol = contenido[z]

    if idioma[z] == "en":
        traducido_ingles = contenido[z]
    elif detect_language(contenido2[z]) == "English":
        traducido_ingles = contenido2[z]
    else:
        try:
            traducido_ingles = traductor_ingles.translate(text = contenido[z])
        except:
            traducido_ingles = None
            print("error")
        
   
    
    texto_espanol.append(traducido_espanol)
    texto_ingles.append(traducido_ingles)

df["texto_espanol"] = texto_espanol
df["texto_ingles"] = texto_ingles


## Con este código estarían los textos traducidos e introducidos en la base de datos


## La duración varía dependiendo de la cantidad de los textos, 1,2M de textos en 7 dias